In [139]:
import pandas as pd
import numpy as np
import time
import soup

ModuleNotFoundError: No module named 'soup'

In [2]:
test_players = pd.read_csv('/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/test_players.csv')

In [3]:
test_players.head()

,link,player,team
0,/Players/144175/Show/Fabián-Noguera,Fabián Noguera,Estudiantes
1,/Players/125446/Show/Enzo-Kalinski,Enzo Kalinski,Estudiantes
2,/Players/133458/Show/Facundo-Sánchez,Facundo Sánchez,Estudiantes
3,/Players/129265/Show/Fernando-Zuqui,Fernando Zuqui,Estudiantes
4,/Players/284579/Show/Lucas-Rodríguez,Lucas Rodríguez,Estudiantes


In [137]:
def get_soup(URL,click=False,element=None):
    # Setting options to run headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    
    # Installing temporary Chrome webdriver
    driver = webdriver.Chrome(ChromeDriverManager().install(),
                             options=chrome_options)
    
    # Trying to get URL and refreshing in case of TimeoutException
    try:
        driver.get(URL)
        wait = WebDriverWait(driver, 10)
    except TimeoutException as ex:
        webDriver.navigate().refresh()
        
    # Entering if get_soups needs click 
    if click:
        
        # In case of being banned, tries again
        if len(driver.find_elements_by_css_selector("[name=ROBOTS]")) > 0:
            try:
                driver.get(URL)
                wait = WebDriverWait(driver, 10)
            except TimeoutException as ex:
                webDriver.navigate().refresh()
        
        # Collecting path and clicking
        tab_click = driver.find_element_by_xpath(element)
        tab_click.click()
        
        # Sleeps for a while to get te refreshed code
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source)
        driver.close()
        
    # Second case if of not needing clicking
    else:
        if len(driver.find_elements_by_css_selector("[name=ROBOTS]")) > 0:
            try:
                driver.get(URL)
                wait = WebDriverWait(driver, 10)
            except TimeoutException as ex:
                webDriver.navigate().refresh()
        soup = BeautifulSoup(driver.page_source)
        driver.close()
        
    return soup

## Whoscored scrapping

**Obtaining players basic information**

In [36]:
def extract_player_info_block(players_df):
    
    links = players_df.link
    teams = players_df.team
    
    player_info = {'name': [],
                  'position': [],
                  'nationality': [],
                  'height': [],
                  'weight': [],
                  'team': []} 

    for each_link, each_team in zip(links,teams):
        URL = 'https://www.whoscored.com' + each_link
        print(URL)
        soup = get_soup(URL)
        text = ""
        for each in soup.find_all('dl', attrs={'class':'player-info-block'}):
            try:
                text = text + str(each)
            except:
                None
        
        name = re.findall(r'((?<=Name:</dt>\n<dd>).+(?=</dd>\n</dl><dl class))', text)
        height = re.findall(r'((?<=Height:</dt>\n<dd>).+(?=cm</dd>))', text)
        position = re.findall(r'((?<=Positions:</dt>\n<dd>\n<ul>\n<li>).*(?= |</li>))', text)
        nation = re.findall(r'((?<="iconize iconize-icon-left">\n).+(?=\n))', text)
        weight = re.findall(r'((?<=Weight:</dt>\n<dd>).+(?=kg))', text)
        
        player_info['name'].append(name[0]) if len(name) >0 else player_info['name'].append(np.nan)
        player_info['position'].append(position[0].strip()) if len(position) >0 else player_info['position'].append(np.nan)
        player_info['nationality'].append(nation[0].strip()[0:3].upper()) if len(nation) >0 else player_info['nationality'].append(np.nan)
        player_info['height'].append(height[0]) if len(height) >0 else player_info['height'].append(np.nan)
        player_info['weight'].append(weight[0]) if len(weight) >0 else player_info['weight'].append(np.nan)
        player_info['team'].append(each_team)

    player_info_df = pd.DataFrame(player_info)
    
    return player_info_df

In [37]:
test = extract_player_info_block(test_players.iloc[0:5,:])

https://www.whoscored.com/Players/144175/Show/Fabián-Noguera

Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
https://www.whoscored.com/Players/125446/Show/Enzo-Kalinski

Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
https://www.whoscored.com/Players/133458/Show/Facundo-Sánchez

Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
https://www.whoscored.com/Players/129265/Show/Fernando-Zuqui

Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
https://www.whoscored.com/Players/284579/Show/Lucas-Rodríguez

Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac

In [38]:
test.to_csv(r'/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/demographic_info.csv',index=False)

**Obtaining players defensive statistics**

In [104]:
def extract_player_defensive(players_df):
    
    links = players_df.link
    teams = players_df.team
    
    player_info = {'interception': [],
                   'tackles': [],
                   'fouls': [],
                   'offsideswon': [],
                   'clearence': [],
                   'wasdribbled': [],
                   'blocks': [],
                   'owngoals': [],
                   'team': [],
                   'link': []
                  } 

    for each_link, each_team in zip(links,teams):
        if "Show" in each_link:
            new_link = each_link.replace("Show","History")
            URL = 'https://www.whoscored.com' + new_link
            soup = get_soup(URL, click=True, element='//*[@id="player-tournament-stats-options"]/li[2]/a')
            print(URL)

            amount = len([each.text.strip() for each in soup.find_all('td', attrs={'class':'tacklePerGame'})])

            player_info['interception'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'interceptionPerGame'})]

            player_info['tackles'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'tacklePerGame'})]

            player_info['fouls'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'foulsPerGame'})]

            player_info['offsideswon'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'offsideWonPerGame'})]

            player_info['clearence'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'clearancePerGame'})]

            player_info['wasdribbled'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'wasDribbledPerGame'})]

            player_info['blocks'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'outfielderBlockPerGame'})]

            player_info['owngoals'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'goalOwn'})]

            player_info['link'] += [each_link for x in range(0,amount)]

            player_info['team'] += [each_team for x in range(0,amount)]

    return player_info

In [105]:
test_defensive = extract_player_defensive(test_players.iloc[np.r_[0:1]])


Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
https://www.whoscored.com/Players/144175/History/Fabián-Noguera


In [106]:
test_defensive

{'interception': ['1.5', '1', '1.4', '0.4'],
 'tackles': ['0.9', '2', '1.6', '0.6'],
 'fouls': ['1.2', '1', '0.6', '1.1'],
 'offsideswon': ['0.6', '-', '0.1', '0.3'],
 'clearence': ['7.9', '4', '7.1', '2.7'],
 'wasdribbled': ['0.3', '1.3', '0.6', '0.4'],
 'blocks': ['0.7', '0.7', '1', '-'],
 'owngoals': ['-', '-', '-', '-'],
 'team': ['Estudiantes', 'Estudiantes', 'Estudiantes', 'Estudiantes'],
 'link': ['/Players/144175/Show/Fabián-Noguera',
  '/Players/144175/Show/Fabián-Noguera',
  '/Players/144175/Show/Fabián-Noguera',
  '/Players/144175/Show/Fabián-Noguera']}

In [107]:
test_df = pd.DataFrame(test_defensive)

In [108]:
test_df

,interception,tackles,fouls,offsideswon,clearence,wasdribbled,blocks,owngoals,team,link
0,1.5,0.9,1.2,0.6,7.9,0.3,0.7,-,Estudiantes,/Players/144175/Show/Fabián-Noguera
1,1,2,1,-,4,1.3,0.7,-,Estudiantes,/Players/144175/Show/Fabián-Noguera
2,1.4,1.6,0.6,0.1,7.1,0.6,1,-,Estudiantes,/Players/144175/Show/Fabián-Noguera
3,0.4,0.6,1.1,0.3,2.7,0.4,-,-,Estudiantes,/Players/144175/Show/Fabián-Noguera


**Obtaining players summary statistics**

In [40]:
def extract_player_summary(players_df):
    
    links = players_df.link
    teams = players_df.team
    
    player_info = {'link': [],
                   'team': [],
                   'mins': [],
                   'goals': [],
                   'assist': [],
                   'yellow': [],
                   'red': [],
                   'shots': [],
                   'passes': [],
                   'aerial': [],
                   'mom': [],
                   'rating': [],
                   'season': []
                  } 

    for each_link, each_team in zip(links,teams):
        if "Show" in each_link:
            new_link = each_link.replace("Show","History")
            URL = 'https://www.whoscored.com' + new_link
            soup = get_soup(URL)
            print(URL)
            
            amount = len([each.text.strip() for each in soup.find_all('td', attrs={'class':'rank tournament'})])

            player_info['mins'] += test_defensive = extract_player_defensive(test_players.iloc[np.r_[0:1]])

test_defensive

test_df = pd.DataFrame(test_defensive)

test_df

            player_info['goals'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'goal'})]

            player_info['assist'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'assistTotal'})]

            player_info['yellow'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'yellowCard'})]

            player_info['red'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'redCard'})]

            player_info['shots'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'shotsPerGame'})]

            player_info['passes'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'passSuccess'})]

            player_info['aerial'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'aerialWonPerGame'})]

            player_info['mom'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'manOfTheMatch'})]

            player_info['rating'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'rating'})[0:-1]]

            player_info['season'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'rank tournament'})]

            player_info['link'] += [each_link for x in range(0,amount)]

            player_info['team'] += [each_team for x in range(0,amount)]

    return player_info

In [41]:
soup_ = extract_player_summary(test_players.iloc[np.r_[0:5,50:55,100:105]])


Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
https://www.whoscored.com/Players/144175/History/Fabián-Noguera

Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
https://www.whoscored.com/Players/125446/History/Enzo-Kalinski

Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
https://www.whoscored.com/Players/133458/History/Facundo-Sánchez

Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
https://www.whoscored.com/Players/129265/History/Fernando-Zuqui

Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
https://www.whoscored.com/Players/

In [42]:
test_df = pd.DataFrame(soup_)

In [43]:
test_df.head()

,link,team,mins,goals,assist,yellow,red,shots,passes,aerial,mom,rating,season
0,/Players/144175/Show/Fabián-Noguera,Estudiantes,1035,2,-,2,-,1.2,71.4,4.8,1,7.23,2018/2019
1,/Players/144175/Show/Fabián-Noguera,Estudiantes,186,-,-,-,-,0.3,74.8,2,-,6.53,2017/2018
2,/Players/144175/Show/Fabián-Noguera,Estudiantes,514,-,1,-,-,0.9,84,2.7,1,6.94,2017
3,/Players/144175/Show/Fabián-Noguera,Estudiantes,305,1,-,1,-,0.4,77.9,2.9,1,6.48,2016
4,/Players/133458/Show/Facundo-Sánchez,Estudiantes,2013,-,2,2,-,0.4,67.7,1.9,3,7.03,2018/2019


In [44]:
test_df.to_csv(r'/Users/gconzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/players_basic_info_test.csv',index=False)

**Obtaining players offensive statistics**

In [133]:
def extract_player_offensive(players_df):
    
    links = players_df.link
    teams = players_df.team
    
    player_info = {'keypasses': [],
                   'dribbles': [],
                   'fouled': [],
                   'offsidesdone': [],
                   'disposeseed': [],
                   'badcontrol': [],
                   'team': [],
                   'link': []
                  } 

    for each_link, each_team in zip(links,teams):
        if "Show" in each_link:
            new_link = each_link.replace("Show","History")
            URL = 'https://www.whoscored.com' + new_link
            soup = get_soup(URL, click=True, element='//*[@id="player-tournament-stats-options"]/li[3]/a')
            print(URL)

            amount = len([each.text.strip() for each in soup.find_all('td', attrs={'class':'keyPassPerGame'})])

            player_info['keypasses'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'keyPassPerGame'})]

            player_info['dribbles'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'dribbleWonPerGame'})]

            player_info['fouled'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'foulGivenPerGame'})]

            player_info['offsidesdone'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'offsideGivenPerGame'})]

            player_info['disposeseed'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'dispossessedPerGame'})]

            player_info['badcontrol'] += [each.text.strip() for each in soup.find_all('td', attrs={'class':'turnoverPerGame'})]

            player_info['link'] += [each_link for x in range(0,amount)]

            player_info['team'] += [each_team for x in range(0,amount)]

    return player_info

In [138]:
test_offensive = extract_player_offensive(test_players.iloc[np.r_[0:1]])


Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
https://www.whoscored.com/Players/144175/History/Fabián-Noguera


In [128]:
test_offensive

{'keypasses': ['-', '-', '0.1', '0.1'],
 'dribbles': ['0.3', '-', '-', '0.1'],
 'fouled': ['0.3', '0.3', '0.3', '0.4'],
 'offsidesdone': ['0.3', '-', '-', '0.1'],
 'disposeseed': ['0.2', '0.3', '-', '0.1'],
 'badcontrol': ['0.3', '1.3', '0.4', '0.4'],
 'team': ['Estudiantes', 'Estudiantes', 'Estudiantes', 'Estudiantes'],
 'link': ['/Players/144175/Show/Fabián-Noguera',
  '/Players/144175/Show/Fabián-Noguera',
  '/Players/144175/Show/Fabián-Noguera',
  '/Players/144175/Show/Fabián-Noguera']}

In [129]:
test_df_of = pd.DataFrame(test_offensive)

In [130]:
test_df_of

,keypasses,dribbles,fouled,offsidesdone,disposeseed,badcontrol,team,link
0,-,0.3,0.3,0.3,0.2,0.3,Estudiantes,/Players/144175/Show/Fabián-Noguera
1,-,-,0.3,-,0.3,1.3,Estudiantes,/Players/144175/Show/Fabián-Noguera
2,0.1,-,0.3,-,-,0.4,Estudiantes,/Players/144175/Show/Fabián-Noguera
3,0.1,0.1,0.4,0.1,0.1,0.4,Estudiantes,/Players/144175/Show/Fabián-Noguera
